# Checking the straight-and-spaced condition

This notebook gives an implementation of the straight-and-spaced check for Anosov representations described in [LINK GOES HERE].

**Before attempting to run the notebook, please consult the README.md file for instructions on how to install required tools.**

In [1]:
import time

import numpy as np

from geometry_tools import representation, utils
from geometry_tools.automata import fsa

# "symmetric" is a local module which we use for some symmetric space computation
import symmetric

### Defining the representation

The specific representation we will test is a Hitchin representation in the Fuchsian locus. Each generator is of the form $R_{\pi/4}^n \cdot L \cdot R_{\pi/4}^{-n}$ for $n = 0, 1, 2, 3$, where $L$ is a loxodromic isometry with axis passing through a point $x_0$ in $\mathbb{H}^2$, and and $R_\theta$ is the rotation in $\mathbb{H}^2$ of angle $\theta$ about $x_0$.

In [2]:
t = 2*np.arccosh(1/np.tan(np.pi/8))
c = np.cosh(t)
s = np.sinh(t)

angle = np.pi / 4

rotation = np.array([
    [np.cos(angle), np.sin(angle), 0],
    [-np.sin(angle), np.cos(angle), 0],
    [0, 0, 1]
])

loxodromic = np.array([[c,0,s],[0,1,0],[s,0,c]])

rep = representation.Representation()
# define r and a; the other generators are conjugates of a by powers of r
rep["a"] = loxodromic
rep["r"] = rotation

rep["b"] = rep["raR"]
rep["c"] = rep["rraRR"]
rep["d"] = rep["rrraRRR"]

As a sanity check, we can test to see if the relation for this surface group holds for this representation (at least up to numerical error). 

In [3]:
relation = "adCbADcB"
rep[relation] # result should be close to the 3x3 identity matrix

array([[ 1.00000000e+00,  6.69855870e-12,  1.74880846e-11],
       [ 9.67322501e-12,  1.00000000e+00, -1.06107200e-11],
       [-9.55834110e-12,  3.59444935e-12,  1.00000000e+00]])

### Setting up the computation

Checking the "straight" part of the straight-and-spaced condition involves computing "$\zeta$-angles," which depend on a regular direction $\zeta$ in the standard Cartan subalgebra of SL(3, R) (this Cartan subalgebra is just the space of traceless diagonal matrices).

In [4]:
zeta = np.diag(np.array([2,-1,-1]));
i_zeta = np.diag(np.array([1,1,-2]));

We need to load a finite-state automaton which accepts unique geodesic representatives for each element in the surface group. This automaton was originally generated by the [kbmag](https://gap-packages.github.io/kbmag/) program.

In [5]:
automaton = fsa.load_kbmag_file("surface_automaton.wa")

Most of the iteration in the algorithm is handled inside of giant numpy arrays (so that we have fast C loops instead of slow Python loops). However, we need to cap the size of the numpy arrays we will work with, because otherwise we will allocate unreasonable amounts of memory.

In [6]:
# ensures that we will not try and create ndarrays with more than 1000 * 1000 points in them
ARR_SIZE_CAP = 1000

### Running the computation

In [7]:
# we will run the check on words of length 10 (= 5*2)
length = 5

# dictionaries to hold memoized computations as we explore the automaton
pre_memos = {}
post_memos = {}

In [8]:
# initial worst angles/spacing
worst_z_angle = 1.0
worst_iz_angle = 1.0
worst_dist = 1e20 # this should be ~infinity

In [9]:
starttime = time.time()

for vertex in automaton.vertices():
    # for each vertex v of the automaton:
    # 1. get an array of elements of length l coming from paths ending at v
    # 2. get an array of elements of length l coming from paths starting at v
    prefix = rep.automaton_accepted(automaton, length, end_state=vertex,
                                    maxlen=False, precomputed=pre_memos)
    postfix = rep.automaton_accepted(automaton, length, start_state=vertex,
                                      maxlen=False, precomputed=post_memos)

    if len(prefix) == 0 or len(postfix) == 0:
        continue

    # cut up the prefix and postfix lists into fixed-length chunks,
    # and run the straight-and-spaced check on them
    n = len(prefix)
    m = len(postfix)
    n_segments = int(n / ARR_SIZE_CAP) + 1
    m_segments = int(m / ARR_SIZE_CAP) + 1
    for i in range(n_segments):
        for j in range(m_segments):

            # compute points in the symmetric space coming from the prefix/postfix isometries,
            # as well as the midpoints on the geodesics from these points to the origin.
            prefix_elts = np.linalg.inv(
                    prefix[i * ARR_SIZE_CAP:min((i + 1) * ARR_SIZE_CAP, n)]
            )
            postfix_elts = postfix[j * ARR_SIZE_CAP:min((j + 1) * ARR_SIZE_CAP, m)]
            
            p1 = symmetric.SymmetricPoint.from_translation(prefix_elts)
            p2 = symmetric.SymmetricPoint.from_translation(postfix_elts)
            
            m1 = p1.origin_midpoint()
            m2 = p2.origin_midpoint()
            
            # filter out points where an eigenvalue is zero since they cause numerical issues / crashes
            m1_nonzero = np.all(m1.a != 0.0, axis=-1)
            m2_nonzero = np.all(m2.a != 0.0, axis=-1)            
            m1 = m1[m1_nonzero]
            m2 = m2[m2_nonzero]

            # also compute the "backwards" midpoints coming from the extended geodesic segments
            m1_inv = m1.invert_by_transvection()
            m2_inv = m2.invert_by_transvection()

            # to compute relevant angles, we need to translate one of our midpoints to 
            # the origin and see where the other one ends up 
            m1_inv_m2 = symmetric.apply_isometry(m1.to_origin(), m2, 
                                                 broadcast="pairwise_reversed")
            m2_inv_m1 = symmetric.apply_isometry(m2.to_origin(), m1, 
                                                 broadcast="pairwise_reversed")

            # compute distances and zeta-angles
            dists = m1_inv_m2.distance_to_origin()
            zeta_angle = symmetric.zeta_angle_from_origin(m2_inv, m2_inv_m1, zeta, 
                                             broadcast="elementwise")
            iota_zeta_angle = symmetric.zeta_angle_from_origin(m1_inv, m1_inv_m2, i_zeta,
                                                  broadcast="elementwise")
            
            worst_dist = min(worst_dist, np.min(dists))
            worst_z_angle = min(worst_z_angle, np.min(zeta_angle))
            worst_iz_angle = min(worst_iz_angle, np.min(iota_zeta_angle))

            print(
                "vertex: {} / {}, chunk {} / {} ({} group elements)".format(
                    vertex, len(automaton.vertices()),
                    i * m_segments + j + 1, n_segments * m_segments,
                    len(prefix_elts) * len(postfix_elts)
            ))

timedelta = time.time() - starttime
print(f"Complete, time was {timedelta} seconds.")
print(f"Worst spacing: {worst_dist}")
print(f"Worst zeta_angle: {worst_z_angle}")
print(f"Worst iota_zeta angle: {worst_iz_angle}")

vertex: 2 / 37, chunk 1 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 2 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 3 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 4 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 5 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 6 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 7 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 8 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 9 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 10 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 11 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 12 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 13 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 14 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 15 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 16 / 34 (1000000 group elements)
vertex: 2 / 37, chunk 17 / 34 (703000 group elements)
vertex: 2 / 37, chunk 18 / 34 (709000 group elements)
vertex: 2 / 37, chunk